Covid 19 CT scan Image classification with VGG19 pretrained model.

Datasets: 2 public datasets stored in the folders CT_DATA (CT_COVID and CT_NonCovid) and SARS-COV-2-DATA.

DATA-Split folder : initially used to split data from CT-DATA, but will not be used here in the final.


In [1]:
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

In [2]:
cov_path = "/content/drive/MyDrive/Colab Notebooks/CTCov/CT_Data/CT_COVID/"
norm_path = "/content/drive/MyDrive/Colab Notebooks/CTCov/CT_Data/CT_NonCOVID/"

In [3]:
IMAGE_SIZE=[224,224]
batch_size=32
#define paths
covid_path = '/content/drive/MyDrive/Colab Notebooks/CTCov/SARS-COV-2-Data/COVID'
noncovid_path = '/content/drive/MyDrive/Colab Notebooks/CTCov/SARS-COV-2-Data/NON-COVID'

# Use glob to grab images from path .jpg or jpeg
cov_files = glob(cov_path + '/*')
norm_files = glob(norm_path + '/*')

covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [4]:
# Fetch Images and Class Labels from Files
cov_labels = []
norm_labels = []

cov_images=[]
norm_images=[]

for i in range(len(cov_files)):
  image = cv2.imread(cov_files[i]) # read file 
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # arrange format as per keras
  image = cv2.resize(image,(224,224)) # resize as per model
  cov_images.append(image) # append image
  cov_labels.append('CT_COVID') #append class label
  
for i in range(len(norm_files)):
  image = cv2.imread(norm_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  norm_images.append(image)
  norm_labels.append('CT_NonCOVID')

In [5]:
# Fetch Images and Class Labels from Files
covid_labels = []
noncovid_labels = []

covid_images=[]
noncovid_images=[]

for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i]) # read file 
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # arrange format as per keras
  image = cv2.resize(image,(224,224)) # resize as per model
  covid_images.append(image) # append image
  covid_labels.append('CT_COVID') #append class label
  
for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  noncovid_images.append(image)
  noncovid_labels.append('CT_NonCOVID')

In [6]:
covid_images = np.concatenate((covid_images, cov_images), axis=0)
noncovid_images = np.concatenate((noncovid_images, norm_images),axis=0)

In [7]:
covid_labels = np.concatenate((covid_labels, cov_labels), axis=0)
noncovid_labels = np.concatenate((noncovid_labels, norm_labels),axis=0)

In [8]:
len(covid_images)

1598

# Median Blur

In [9]:
covid_images = [cv2.medianBlur(img, 5) for img in covid_images]

In [10]:
noncovid_images = [cv2.medianBlur(img, 5) for img in noncovid_images]

In [11]:
covid_images = np.array(covid_images)/255
noncovid_images = np.array(noncovid_images)/255

In [12]:
# Split into training and testing sets for both types of images
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(
    covid_images, covid_labels, test_size=0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(
    noncovid_images, noncovid_labels, test_size=0.2)

# Merge sets for both types of images
X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)

# Make labels into categories - either 0 or 1, for our model
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)


# ResNet50 model

In [13]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3, shuffle=True)
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

# Building Model
acc_per_fold = []
loss_per_fold = []
fold_no = 1

resnet = ResNet50(weights="imagenet", include_top=False,
  input_tensor=Input(shape=(224, 224, 3)))

for layer in resnet.layers:
      layer.trainable = False

train_aug = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True
  )

for train, test in kfold.split(inputs, targets):

  outputs = resnet.output
  outputs = Flatten(name="flatten")(outputs)
  outputs = Dropout(0.5)(outputs)
  outputs = Dense(2, activation="softmax")(outputs)

  model = Model(inputs=resnet.input, outputs=outputs)


  model.compile(
          loss='categorical_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy']
  )

  

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  model.fit(train_aug.flow(inputs[train], targets[train], batch_size=batch_size),
                    verbose=1,
                    epochs=100)
  
  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
68/68 [==============================] - 24s 292ms/step - loss: 1.5330 - accuracy: 0.5128
Epoch 2/100
68/68 [==============================] - 20s 292ms/step - loss: 1.1194 - accuracy: 0.5398
Epoch 3/100
68/68 [==============================] - 20s 289ms/step - loss: 1.0461 - accuracy: 0.5565
Epoch 4/100
68/68 [==============================] - 20s 291ms/step - loss: 1.3497 - accuracy: 0.5295
Epoch 5/100
68/68 [==============================] - 20s 291ms/step - loss: 1.1481 - accuracy: 0.5458
Epoch 6/100
68/68 [==============================] - 20s 291ms/step - loss: 1.1349 - accuracy: 0.5654
Epoch 7/100
68/68 [==============================] - 20s 292ms/step - loss: 1.5016 - accuracy: 0.5393
Epoch 8/100
68/68 [==============================] - 21s 301ms/step - loss: 1.1990 - accuracy: 0.5696
Epoch 9/100
68/68 [==============================] - 20s 301ms/step - loss: 1.0389 - ac

In [14]:
import pandas as pd
metrics = pd.DataFrame(acc_per_fold)
metrics = metrics.rename(columns={0:'accuracy'})
loss_df = pd.DataFrame(loss_per_fold)

In [15]:
metrics.head()

,accuracy
0,73.767442
1,71.534884
2,74.767226


In [16]:
metrics_df = metrics.join(loss_df.rename(columns={0:'loss'}))

In [17]:
metrics_df.head()

,accuracy,loss
0,73.767442,0.704190
1,71.534884,0.619324
2,74.767226,0.646786


In [18]:
report_csv_file = '/content/drive/MyDrive/Colab Notebooks/CTCov/Fold/ResNet50/res_med_blur_class_report.csv'
with open(report_csv_file, mode='w') as f:
    metrics_df.to_csv(f)